In [13]:
#ライブラリ
import tweepy
import nltk
import stanza
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [14]:
#Twitter API config読み込み
config = pd.read_csv("./config.csv")

In [15]:
# Twitter API config
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [16]:
# Authenticate
auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [17]:
# キーワードからツイートを取得
s='btc min_faves:1 min_retweets:1 -filter:retweets'
tweets = tweepy.Cursor(api.search,  
                        count=10,
                        q=s,
                        exclude_replies=True,
                        tweet_mode='extended',
                        show_user=True,
                        lang = 'en',
                        since='2021-07-20',
                        until='2021-07-21',
                        ).items(1000);

In [18]:
#ツイートのリスト
t_data = []
for tweet in tweets:
    t_data.append([tweet.user.name,'@'+tweet.user.screen_name,tweet.user.friends_count,tweet.user.followers_count,tweet.full_text.replace('\n',''),tweet.favorite_count,tweet.retweet_count,tweet.created_at])

In [19]:
#処理前ツイート出力
df = pd.DataFrame(data=t_data, columns=['Name','User Name','Following','Followers','Tweet','Favorite','RT','created at'])
df.to_csv("original_tweet.csv")

In [20]:
#処理前ツイート先頭10件
df.head(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT,created at
0,En Bloc Press,@en_bloc_press,77,5798,Imagine advocating for fiat and cash currencie...,16,1,2021-07-20 23:59:52
1,luna || crypto to gcash ~ 3% fee,@gelendigkeit,1405,1697,"Hello, I do trades po 🍾ʚ BNB/BUSD to gcash ʚ U...",1,1,2021-07-20 23:59:19
2,T-Plex,@tplexapp,38,1060,I never thought I would have said that once in...,25,1,2021-07-20 23:58:57
3,Jonny Moe,@JonnyMoeTrades,592,21696,$BTC first daily close under $30k since New Ye...,22,1,2021-07-20 23:58:53
4,Gaurav Tripathi,@CryptoTripathi,138,3402,"The markets are still frothy, things are out o...",3,1,2021-07-20 23:58:45
5,Trent Richards,@trentrichards1,597,616,July 2021 Team AMA - Part 1Metrix CEO @trentri...,32,17,2021-07-20 23:57:32
6,pleb_Greg Foss 🇨🇦 / 🇸🇻🌋,@FossGregfoss,1056,26015,I have never been so driven to rectify a wrong...,440,38,2021-07-20 23:56:19
7,Kripto Aksiyon,@KriptoAksiyon,332,2643,🦅 Our analysis of #bitcoin coin by 1 hourly ti...,9,4,2021-07-20 23:55:19
8,iHodl 🟩⚡️⚡️,@iTweet14670008,667,349,"@eric_lombrozo @gillesCadignan Yes, Bitcoin wi...",21,2,2021-07-20 23:53:44
9,Sgt. Michael Sykes,@sgt_sykes,232,42,@nayibbukele Since you came to the agenda with...,7,2,2021-07-20 23:53:42


In [21]:
#処理前ツイート末尾10件
df.tail(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT,created at
990,Crypto Gems Hunter,@cryptogems5561,76,93,@CyrptoSalvador @davidgokhshtein @TheRealKatyK...,8,3,2021-07-20 18:47:08
991,CryptoAmca,@CryptoAmca58,520,381,@cryptomanran @GarethSoloway You wanna sleep c...,17,11,2021-07-20 18:46:48
992,Crypto Rain,@rain_turk,260,138,@earn_finance #EarnXFinance #EarnX $EarnX #Bin...,1,1,2021-07-20 18:46:36
993,Britcoin Boomer,@bruce_pullman,575,1753,It is not looking seriously bearish. Having ma...,17,1,2021-07-20 18:46:18
994,💐💐💐 Guru 💐💐💐,@CoinGurruu,575,16539,Bought 25 BTC because @michael_saylor said $BT...,70,1,2021-07-20 18:46:16
995,CryptoGnomie,@CryptoGnomie,1404,1733,"I dunno who ya'll follow, I don't know who's t...",5,2,2021-07-20 18:45:56
996,Salam Salemy,@SalamSalemy,207,37,⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡King of DeFi offers multiple sta...,2,1,2021-07-20 18:45:48
997,MEHMET RİGBY,@shmsq20,45,4,@CyrptoSalvador @davidgokhshtein @TheRealKatyK...,16,2,2021-07-20 18:45:24
998,ιναη™🍥,@CryptoNinetail,208,115,@cryptomanran @GarethSoloway #Tikitoken where ...,17,11,2021-07-20 18:45:04
999,฿izWiz📡👽👉👌🌏,@ProfFaustus,341,640,Will the #ETH market cap be more than HALF tha...,3,1,2021-07-20 18:45:01


In [22]:
#ツイート前処理
def cleanUpTweet(txt):
    # Remove mentions
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt)
    return txt

In [23]:
#lemmatization
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def lemma(txt):
    s = ''
    doc = nlp(txt)
    
    for sent in doc.sentences:
        for word in sent.words:
            s += word.lemma + ' '
    
    return s

2021-07-23 16:17:23 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-07-23 16:17:23 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-07-23 16:17:23 INFO: Use device: cpu
2021-07-23 16:17:23 INFO: Loading: tokenize
2021-07-23 16:17:23 INFO: Loading: pos
2021-07-23 16:17:24 INFO: Loading: lemma
2021-07-23 16:17:24 INFO: Done loading processors!


In [24]:
#処理後ツイート出力
df['treated Tweet'] = df['Tweet'].apply(lemma)
df.to_csv("treated_tweet.csv")

In [25]:
#処理後ツイート先頭10件
df.head(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT,created at,treated Tweet
0,En Bloc Press,@en_bloc_press,77,5798,Imagine advocating for fiat and cash currencie...,16,1,2021-07-20 23:59:52,imagine advocate for fiat and cash currency an...
1,luna || crypto to gcash ~ 3% fee,@gelendigkeit,1405,1697,"Hello, I do trades po 🍾ʚ BNB/BUSD to gcash ʚ U...",1,1,2021-07-20 23:59:19,"hello , I do trade po 🍾ʚ BNB / BUSD to gcash ʚ..."
2,T-Plex,@tplexapp,38,1060,I never thought I would have said that once in...,25,1,2021-07-20 23:58:57,I never think I would have say that once in my...
3,Jonny Moe,@JonnyMoeTrades,592,21696,$BTC first daily close under $30k since New Ye...,22,1,2021-07-20 23:58:53,$ BTC first daily close under $ 30 k since New...
4,Gaurav Tripathi,@CryptoTripathi,138,3402,"The markets are still frothy, things are out o...",3,1,2021-07-20 23:58:45,"the market be still frothy , thing be out of p..."
5,Trent Richards,@trentrichards1,597,616,July 2021 Team AMA - Part 1Metrix CEO @trentri...,32,17,2021-07-20 23:57:32,July 2021 team AMA - part 1 Metrix ceo @trentr...
6,pleb_Greg Foss 🇨🇦 / 🇸🇻🌋,@FossGregfoss,1056,26015,I have never been so driven to rectify a wrong...,440,38,2021-07-20 23:56:19,I have never be so drive to rectify a wrong ( ...
7,Kripto Aksiyon,@KriptoAksiyon,332,2643,🦅 Our analysis of #bitcoin coin by 1 hourly ti...,9,4,2021-07-20 23:55:19,🦅 we analysis of # bitcoin coin by 1 hourly ti...
8,iHodl 🟩⚡️⚡️,@iTweet14670008,667,349,"@eric_lombrozo @gillesCadignan Yes, Bitcoin wi...",21,2,2021-07-20 23:53:44,"@eric_lombrozo @gilles Cadignan yes , Bitcoin ..."
9,Sgt. Michael Sykes,@sgt_sykes,232,42,@nayibbukele Since you came to the agenda with...,7,2,2021-07-20 23:53:42,@nayibbukele since you come to the agenda with...


In [26]:
#処理前ツイート末尾10件
df.tail(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT,created at,treated Tweet
990,Crypto Gems Hunter,@cryptogems5561,76,93,@CyrptoSalvador @davidgokhshtein @TheRealKatyK...,8,3,2021-07-20 18:47:08,@cyrptosalvador @davidgokhshtein @ TheReal Kat...
991,CryptoAmca,@CryptoAmca58,520,381,@cryptomanran @GarethSoloway You wanna sleep c...,17,11,2021-07-20 18:46:48,@cryptomanran @ GarethSoloway you wanna sleep ...
992,Crypto Rain,@rain_turk,260,138,@earn_finance #EarnXFinance #EarnX $EarnX #Bin...,1,1,2021-07-20 18:46:36,@earn_finance # EarnXFinance # EarnX $ EarnX #...
993,Britcoin Boomer,@bruce_pullman,575,1753,It is not looking seriously bearish. Having ma...,17,1,2021-07-20 18:46:18,it be not look seriously bearish . have make t...
994,💐💐💐 Guru 💐💐💐,@CoinGurruu,575,16539,Bought 25 BTC because @michael_saylor said $BT...,70,1,2021-07-20 18:46:16,buy 25 BTC because @michael_saylor say $ BTC f...
995,CryptoGnomie,@CryptoGnomie,1404,1733,"I dunno who ya'll follow, I don't know who's t...",5,2,2021-07-20 18:45:56,"I dunno who ya'll follow , I do not know who b..."
996,Salam Salemy,@SalamSalemy,207,37,⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡King of DeFi offers multiple sta...,2,1,2021-07-20 18:45:48,⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ King of DeFi offer multiple sta...
997,MEHMET RİGBY,@shmsq20,45,4,@CyrptoSalvador @davidgokhshtein @TheRealKatyK...,16,2,2021-07-20 18:45:24,@cyrptosalvador @davidgokhshtein @ TheReal Kat...
998,ιναη™🍥,@CryptoNinetail,208,115,@cryptomanran @GarethSoloway #Tikitoken where ...,17,11,2021-07-20 18:45:04,@cryptomanran @ GarethSoloway # Tikitoken wher...
999,฿izWiz📡👽👉👌🌏,@ProfFaustus,341,640,Will the #ETH market cap be more than HALF tha...,3,1,2021-07-20 18:45:01,will the # ETH market cap be more than half th...


In [ ]:
#VADER感情分析器インポート
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

In [ ]:
#感情出力関数
def getTextPolarity(txt):
    return vader_analyzer.polarity_scores(txt)
def getTextPos(txt):
    return txt.get("pos")
def getTextNeu(txt):
    return txt.get("neu")
def getTextNeg(txt):
    return txt.get("neg")
def getTextCom(txt):
    return txt.get("compound")

def getTextAnalysis(l):
    s = l.get("compound")
    if s < 0:
        return "Negative"
    elif s == 0:
        return "Neutral"
    else:
        return "Positive"

In [ ]:
#表に分析結果を追加
df['Score'] = df['Tweet'].apply(getTextPolarity)
df['neg'] = df['Score'].apply(getTextNeg)
df['neu'] = df['Score'].apply(getTextNeu)
df['pos'] = df['Score'].apply(getTextPos)
df['compound'] = df['Score'].apply(getTextCom)
df['Sentiment'] = df['Score'].apply(getTextAnalysis)

In [ ]:
#感情数値表示
df.drop(columns='Score')

In [ ]:
#結果を表に上書き
df.to_csv("treated_tweet.csv")